# Generative AI: Deploy Your Own Chatbot with Load Balance

NeuralChat is a customizable chat framework designed to create user own chatbot within few minutes on multiple architectures. 
This notebook is used to demonstrate how to deploy your own chatbot with load balance on single/multiple nodes.

# Start Backend Services 💡

Following the `Quick Start Local Server` part of notebook `chatbot_on_intel_cpu.ipynb` to start Chatbot services.

For example, you have two services running on four nodes as below, the `host_ip` and `port` are the real host ip and port numbers of your server.
1. Textchat service
    - http://host_ip_1:port_1/v1/chat/completions
    - http://host_ip_2:port_2/v1/chat/completions
2. Voicechat service
    - http://host_ip_3:port_3/v1/voicechat/completions
    - http://host_ip_4:port_4/v1/voicechat/completions

# Configure Nginx Server 💻
You can either download and install from Nginx source code for additional scalable services such as MySQL, or install directly from apt. 

## Install from Source Code

#### Install
The example download version is Nginx-1.18.0, you can choose to download other versions of different platform [here](https://nginx.org/en/download.html).

In [ ]:
sudo apt-get update
wget http://nginx.org/download/nginx-1.18.0.tar.gz
tar -zxvf nginx-1.18.0.tar.gz
cd nginx-1.18.0

#### Compile and Start Nginx Service

In [ ]:
# compile Makefile
./configure --with-stream --with-stream_ssl_module --with-http_ssl_module --without-http_rewrite_module --without-http_gzip_module --with-openssl-opt="no-deprecated"
# compiling
make CFLAGS="-Wno-error=deprecated-declarations"
# install and start service
sudo make install
sudo /usr/local/nginx/sbin/nginx
# check whether nginx is started
ps -elf | grep nginx

## Install from apt

#### Install
Using apt, you can install nginx with a single command.

In [ ]:
# install nginx
sudo apt install nginx

#### Start Nginx Service

In [ ]:
sudo systemctl start nginx
sudo systemctl status nginx

# Configure Nginx 🔧
First, you need to find the configuration file.

In [ ]:
# In case you install nginx in different path, find the path of nginx.conf first
find / -name nginx.conf
# modify conf file
vim /etc/nginx/nginx.conf

## Single Server
In Nginx configuration file, you need to add `upstream` part and modify `server` part in `http`.

Define your local service in `upstream` with different host ip and port number. (Assume the services are running on local server, so the host ip is `localhost`.)

Specify the listening port number in `server`.
`location` indicates that `/text/chat` will use the upstream `text_chat_service`, and `proxy_pass` specify the Restful API of `/v1/chat/completions`.

http {

    ########################
    # Other Basic Settings #
    ########################

    #########################
    # Load Balance Settings #
    #########################

    upstream text_chat_service {
        server localhost:{port_1};
        server localhost:{port_2};
    }

    ###################
    # Server Settings #
    ###################

    server {
        listen 80;

        location /text/chat {
            proxy_pass http://text_chat_service/v1/chat/completions;
        }
    }
}

## Multiple Server
The difference between single/multiple server is the `upstream` part. `localhost` will be replaced by the real host number.

http {

    ########################
    # Other Basic Settings #
    ########################

    #########################
    # Load Balance Settings #
    #########################

    upstream text_chat_service {
        server {host_ip_1}:{port_1};
        server {host_ip_2}:{port_2};
    }

    ###################
    # Server Settings #
    ###################

    server {
        listen 80;

        location /text/chat {
            proxy_pass http://text_chat_service/v1/chat/completions;
        }
    }
}

## Multiple Services
If you need to support multiple services through one nginx server, add more `upstream` and `location` in nginx.conf.

http {

    ########################
    # Other Basic Settings #
    ########################

    #########################
    # Load Balance Settings #
    #########################

    upstream text_chat_service {
        server {host_ip_1}:{port_1};
        server {host_ip_2}:{port_2};
    }

    upstream voice_chat_service {
        server {host_ip_3}:{port_3};
        server {host_ip_4}:{port_4};
    }

    ###################
    # Server Settings #
    ###################

    server {
        listen 80;

        location /text/chat {
            proxy_pass http://text_chat_service;
        }

        location /voice/chat {
            proxy_pass http://voice_chat_service;
        }
    }
}

# Load Balance 🚀

## Restart Nginx
Restart nginx server to enable load balance service.

In [ ]:
systemctl restart nginx

## Test Load Balance
Assume the host of Nginx server is `198.0.0.1`, test load balance using curl command. (Make sure port number is the same as defined in nginx.conf)

In [ ]:
curl -X POST -H "Content-Type: application/json" -d '{"prompt": "Tell me about Intel Xeon Scalable Processors."}' http://198.0.0.1:80/v1/chat/completions